In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np

device = torch.device("cpu")

In [2]:
import numpy as np
import os
import matplotlib.pyplot as plt
from glob import glob

dirs = sorted(os.listdir("outs"))
list(enumerate(dirs))

[(0, '181001_dense169_3D_bac18'),
 (1, '181002_dense201_3D_bac18'),
 (2, '181003_dense169_gen'),
 (3, '181003_dense169_mot'),
 (4, '181003_dense169_shape'),
 (5, '181004_attvgg_18bac'),
 (6, '181005_dense169_allmulti'),
 (7, '181005_dense169_gramshapemot'),
 (8, '181007_dense169_allmulti'),
 (9, '181008_dense169_gram'),
 (10, '181008_dense169_shape'),
 (11, '181009_genus'),
 (12, '181009_gram'),
 (13, '181009_mot'),
 (14, '181009_shape'),
 (15, '181010_z10to30_aug004_b64'),
 (16, '181010_z10to30_aug00_b64'),
 (17, '181010_z10to30_aug05_b64'),
 (18, '181010_z10to30_aug30_b64'),
 (19, '181012_cascade'),
 (20, '181012_noaug30_z10to30_b64'),
 (21, '181012_noaug50_z10to30_b64'),
 (22, '181013_dp131_b64'),
 (23, '181013_dpn107_b64'),
 (24, '181013_dpn98_b64'),
 (25, '181013_dpn98_z10to40'),
 (26, '181014_dpn107_b64'),
 (27, '181014_dpn107fix_b64'),
 (28, '181014_dpn131fix_b64'),
 (29, '181016_lrelu_z10to30_b64_pre50_nopre100'),
 (30, '181017_newdata_lrelu_z10to30_b64_pre50_nopre100'),
 (31, 

In [3]:
target_dir = dirs[57]
ckpoint_path = sorted(glob("outs/" + target_dir + "/epoch*"))[-1]
ckpoint_path

'outs/181104_base_b64_noaug_valid40/epoch[00189]_acc[0.9333]_test[0.9514].pth.tar'

In [4]:
import torch
from models.Densenet3d_nomaxpool import d169_3d
net = d169_3d(num_classes=18, sample_size=64, sample_duration=96,
              norm="bn", act="lrelu", se=[], af=[])

net = torch.nn.DataParallel(net)
state = torch.load(ckpoint_path)["network"]
net.load_state_dict(state)
model = net.module
model.to(device)
print("Net Load")

Net Load


In [5]:
from visualizes.GradCam import GradCam
# grad_cam = GradCam.get_layer_name(model)

In [6]:
# from visualizes.GradCam_torch import GradCamPyTorch as GradCam
from visualizes.GradCam import GradCam

grad_cam = GradCam(model=model, hooks=["features", "transition1"], device=device)
# Make New model variance

from visualizes.GuidedBackpropReLUModel import GuidedBackpropReLUModel
gb_model = GuidedBackpropReLUModel(model=model, device=device)

In [7]:
from datas.BacLoader import bacLoader
from datas.preprocess3d import TRAIN_AUGS_3D, TEST_AUGS_3D

data_path = "/data2/DW/180930_bac/valid40/"

test_loader = bacLoader(data_path + "test", 1, task="bac", sampler=False,
                        transform=TEST_AUGS_3D, aug_rate=0,
                        num_workers=16, shuffle=False, drop_last=False)

{'Smaltophilia': 10, 'Micrococcus_luteus': 7, 'Escherichia_coli': 4, 'Streptococcus_agalactiae': 14, 'Streptococcus_anginosus': 15, 'Acinetobacter_baumannii': 0, 'Streptococcus_pneumoniae': 16, 'Staphylococcus_aureus': 12, 'Streptococcus_pyogenes': 17, 'Enterococcus_faecalis': 3, 'Bacillus_subtilis': 1, 'Pseudomonas_aeruginosa': 9, 'Staphylococcus_epidermidis': 13, 'Proteus_mirabilis': 8, 'Enterobacter_cloacae': 2, 'Listeria_monocytogenes': 6, 'Klebsiella_pneumoniae': 5, 'Smarcescens': 11}
Dataset Dir :  /data2/DW/180930_bac/valid40/test len :  720


In [11]:
import torch.nn.functional as F
for i, (input_, label_, path) in enumerate(test_loader):
    input_.requires_grad_(True)    
    respond_cam, gdcam, gdcampp, gunhocam, predict = grad_cam.multicam(input_, None)
    
    img = input_[0, 0].detach().numpy()
    
    label = test_loader.dataset.idx_to_class[label_.item()]
    pred = test_loader.dataset.idx_to_class[predict.item()]
    fig = plt.figure(figsize=(20, 20))
    plt.suptitle("File : %s\n Label : %s, pred : %s"%(path, label, pred),
                 fontsize=16, y=0.61)
    
    zdim = 10; cmap="hot"
    ax = plt.subplot(151)
    ax.set_title("image z:%d"%(zdim))
    plt.imshow(img[:, :, zdim],         cmap=cmap)
    ax = plt.subplot(152)
    ax.set_title("respond cam")
    plt.imshow(respond_cam[:, :, zdim], cmap=cmap)
    ax = plt.subplot(153)
    ax.set_title("gradcam")
    plt.imshow(gdcam[:, :, zdim],       cmap=cmap)
    ax = plt.subplot(154)
    ax.set_title("gdcampp")
    plt.imshow(gdcampp[:, :, zdim],     cmap=cmap)
    ax = plt.subplot(155)
    ax.set_title("gunhocam")
    plt.imshow(gunhocam[:, :, zdim],    cmap=cmap)

    # /data2/DW/180930_bac/valid40/test/Acinetobacter_baumannii/Tomo_532nm_suppr_1_sp007_timeMarker_001_data_180905.mat
    save_path = "outs/" + target_dir + "/cams/" + "True/" if label == pred else "False/"
    file_name = path[0][path[0].find("/test/") + 6:] + ".png"
    os.makedirs(os.path.dirname(save_path + file_name), exist_ok=True)
    plt.savefig(save_path + file_name, dpi=fig.dpi)
    print(save_path + file_name)
    
    # plt.show()
    plt.close()
    # cam_img = GradCam.cam_on_image(img, cam_mask)
    
    """
    gb_img = gb_model(input_, index=target_index)
    cam_gb = gb_model.gb_on_cam(cam_mask, gb_img)
    
    plt.figure(figsize=(20, 20))
    plt.suptitle("File : %s\n Label : %s, pred : %s"%(path,
                                                      test_loader.dataset.idx_to_class[label_.item()],
                                                      test_loader.dataset.idx_to_class[predict.item()]),
                 fontsize=16, y=0.61)
    
    plt.subplot(141); plt.imshow(img[:, :, 10], cmap="hot")
    plt.subplot(142); plt.imshow(cam_mask[:, :, 10], cmap="hot")
    
    plt.subplot(143); plt.imshow(gb_img[:, :, 10]);
    plt.subplot(144); plt.imshow(cam_gb[:, :, 10]);
    plt.show()
    """

outs/181104_base_b64_noaug_valid40/cams/True/Acinetobacter_baumannii/Tomo_532nm_suppr_1_sp007_timeMarker_001_data_180905.mat.png
